# SQL Join

###  Содержание <a class="anchor" id=0></a>
- [1. Знакомимся с данными](#1)
- [2. Соединение таблиц по ключу](#2)
- [3. Знакомимся с JOIN](#3)
- [4. Фильтрация и агрегатные функции](#4)
- [5. Способы соединения таблиц](#5)
- [5.1 LEFT JOIN](#5-1)
- [5.2 FULL OUTER JOIN](#5-2)
- [5.3 CROSS JOIN](#5-3)
- [5.4 NATURAL JOIN](#5-4)
- [5.5 Общая логика и виды JOIN'ов](#5-5)
- [6. Итоги](#6)

## Знакомимся с данными <a class="anchor" id=1></a>

[к содержанию](#0)

В этом модуле мы будем работать с таблицами о футбольных матчах и командах.

Таблицы этого модуля, как и все другие в курсе, лежат в схеме sql в [Metabase](http://sql.skillfactory.ru:3000/). Нам понадобятся таблицы `teams` и `matches`.

### Таблица teams:

* `id`	- id команды
* `api_id`	- ключ на таблицу matches
* `long_name`	- полное название команды
* `short_name`	- сокращённое название команды

### Таблица matches с данными о матчах

* `id`	- id матча
* `season`	- сезон
* `date`	- дата матча
* `home_team_api_id`	- api_id домашней команды, ключ на таблицу teams 
* `away_team_api_id`	- api_id гостевой команды, ключ на таблицу teams
* `home_team_goals`	- количество голов домашней команды
* `away_team_goals`	- количество голов гостевой команды

>**Примечание**. Как и в предыдущих модулях, вы можете посмотреть подробный разбор структуры предлагаемых запросов во вкладке **Детализация**.

Напишите запрос, который выведет сезон (`season`), общее количество забитых мячей домашними (`total_home_goals`) и гостевыми (`total_away_goals`) командами.
Отсортируйте по столбцу с сезоном в порядке возрастания.

In [ ]:
select
    season,
    sum(home_team_goals) total_home_goals,
    sum(away_team_goals)  total_away_goals
from sql.matches
group by season
order by season

## Соединение таблиц по ключу <a class="anchor" id=2></a>

[к содержанию](#0)

Чтобы соединить две таблицы между собой, достаточно записать названия таблиц через запятую в разделе `from`. Что произойдёт в таком случае?

Каждая запись, которая есть в таблице teams, будет соединена с каждой записью в таблице matches.

Это действие также называют декартовым произведением таблиц.

<img src=sql_3_img1.png>

Действительно ли это произведение?

Легко проверить! В исходных таблицах teams и matches было 299 и 25083 записей соответственно. Если соединить каждую запись одной таблицы с каждой записью другой, получится 299 * 25083 записей в итоговой таблице.

В данном случае соединение таблиц не даёт практической пользы: мы получили очень много записей, которые никак не можем интерпретировать, потому что команды не соответствуют матчам.

Давайте исправим это. В таблице `teams` есть столбец `api_id`, а таблица `matches` содержит столбцы `home_team_api_id` и `away_team_api_id` — это ключи таблиц, по которым они соединяются.

>**Ключ** — это поле (столбец) в таблице, которое позволяет однозначно идентифицировать запись (строку).

Чтобы соединить таблицы и получить данные о домашней команде по каждому матчу, добавим условие
`where home_team_api_id = api_id.`

In [ ]:
SELECT *
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id

Аналогично можем получить данные о гостевых командах: необходимо изменить условие на
`where away_team_api_id = api_id`.

In [ ]:
SELECT *
FROM
    sql.teams,
    sql.matches
WHERE away_team_api_id = api_id

Вы уже знакомы с ключами по таблице `pokemon` (там в этой роли выступал столбец `id`). Ключи нужны для того, чтобы иметь возможность не перепутать между собой различные записи.

### Ключи бывают двух основных типов:

* `Primary` — **первичный ключ** — служит для идентификации текущей таблицы и, как правило, идёт первым в списке столбцов. Всегда уникален: повторяющихся значений в основной таблице быть не может.
* `Foreign` — **внешний ключ** — представляет собой ссылку на другую таблицу.

Как правило, названия ключей имеют «хвост», который позволяет их идентифицировать: например, `_id`, `_rk`, `_cd`, `_pk` (от `primary_key`), `_fk` (от `foreign_key`) и другие.

>**Обратите внимание!** В данном датасете ключ `api_id` таблицы `teams` может быть использован в разных значениях. Его можно использовать для того, чтобы получить информацию о домашней (`home`) или гостевой (`away`) команде.

Вы могли заметить, что в последних двух запросах получилось очень много столбцов. Как и при работе с одиночной таблицей, мы можем выбирать, какие столбцы соединённой таблицы выводить.

С помощью известного нам запроса получим названия команд, игравших домашние матчи, и счёт матчей.

In [ ]:
SELECT 
    long_name, 
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE home_team_api_id = api_id

Напишите запрос, который выведет таблицу с результатами матчей, содержащую:

* названия гостевых команд (`long_name`);
* количество забитых мячей домашней команды (`home_team_goals`);
* количество забитых мячей гостевой команды (`away_team_goals`).

In [ ]:
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE away_team_api_id = api_id

## Знакомимся с JOIN <a class="anchor" id=3></a>

[к содержанию](#0)

>`JOIN` — это оператор `SQL`, который позволяет соединять таблицы по условию.

In [ ]:
SELECT 
    long_name, /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM    
    sql.teams /*таблица с командами*/
JOIN sql.matches on home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

### СИНТАКСИС

Оператор `JOIN` упрощает процесс соединения таблиц.

Его синтаксис можно представить следующим образом:

In [ ]:
SELECT
    столбец1,
	столбец2,
	...
FROM
	таблица1
JOIN таблица2 ON условие

Порядок присоединения таблиц в данном случае **не важен** — результат будет одинаковым.

С помощью `JOIN` можно соединить и более двух таблиц.

In [ ]:
SELECT
    столбец1,
	столбец2,
	...
FROM
	таблица1
JOIN таблица2 ON условие
JOIN таблица3 ON условие

В таблицах, которые мы соединяем, могут быть одинаковые названия столбцов.

→ К примеру, столбец `id` есть и в таблице `matches`, и в таблице `teams`. Такой запрос не будет обработан.

In [ ]:
SELECT
    id
FROM 
sql.teams
    JOIN sql.matches ON home_team_api_id = api_id

В результате должно появиться сообщение об ошибке вроде такого: "`... column id is ambiguous ...`".

?Что же делать в таком случае?

Можно указать, откуда мы хотим запросить данные, записав название таблицы перед столбцом через точку.

In [ ]:
SELECT
    teams.id /*столбец id таблицы teams*/
FROM 
    sql.teams /*таблица teams*/
JOIN sql.matches ON home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

Можно также выбрать и столбец из таблицы `matches`.

In [ ]:
SELECT
    matches.id /*столбец id таблицы matches*/
FROM 
    sql.teams /*таблица teams*/
JOIN sql.matches ON home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/

Зачастую названия таблиц **слишком длинные**, так что использовать их неудобно.

>Упростить обращение к различным таблицам можно, присвоив им сокращённые названия — **алиасы** (от англ. **alias**).

Синтаксис для указания алиаса такой же, как и для названия столбца.

In [ ]:
SELECT
    столбец1,
	столбец2,
	...
FROM
	таблица1 AS короткое_название_1
JOIN таблица2 AS короткое_название_2 ON условие

### Название записывается без пробелов и операторов.

Если необходимо записать название на русском языке (`"таблица1"`) или с пробелом (`"table 2"`), то алиас можно **обернуть в кавычки**.

>**Важно!** Обращаться по такому алиасу придётся также с помощью кавычек.

Использование таких алиасов считается **плохой практикой** как минимум по двум причинам:

* обращаться с такими алиасами неудобно;
* названия на кириллице можно задать не во всех СУБД.

Вместо алиасов с кавычками рекомендуется давать простое короткое название на латинице, без специальных символов и пробелов. Если таблиц немного и все названия начинаются с разных букв, можно присваивать алиасы по первой букве.

Ключевое слово `as`, как и в названии столбца, можно **опустить в большинстве СУБД**.

In [ ]:
SELECT
    столбец1 новое_название_столбца,
	столбец2 новое_название_столбца,
	...
FROM
	таблица1 короткое_название_1
JOIN таблица2 короткое_название_2 ON условие

Напишите запрос, который выведет два столбца: `id` матча (`match_id`) и `id` домашней команды (`team_id`), — а затем отсортируйте по `id` матча в порядке возрастания значений.

In [ ]:
SELECT
    matches.id match_id,
    teams.id team_id
FROM
    sql.matches
JOIN sql.teams on home_team_api_id = api_id
ORDER BY match_id

Кажется, теперь мы можем соединить таблицы между собой и создать своё табло со счётом матчей, как на спортивных сайтах!

Давайте с помощью запроса SQL получим таблицу, содержащую:

* название домашней команды;
* количество забитых домашней командой голов;
* количество забитых гостевой командой голов;
* название гостевой команды.

In [ ]:
SELECT
    h.long_name "домашняя команда", /*столбец long_name таблицы h*/
    m.home_team_goals "голы домашней команды", /*столбец home_team_goals таблицы m*/
    m.away_team_goals "голы гостевой команды", /*столбец away_team_goals таблицы m*/
    a.long_name "гостевая команда" /*столбец long_name таблицы a*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams h ON m.home_team_api_id = h.api_id /*оператор соединения таблиц; таблица teams с алиасом h; условие: home_team_api_id таблицы m равен api_id таблицы h*/
    JOIN sql.teams a ON m.away_team_api_id = a.api_id /*оператор соединения таблиц; таблица teams с алиасом a; условие: away_team_api_id таблицы m равен api_id таблицы a*/

Напишите запрос, который выведет столбцы: id матча, короткое название домашней команды (home_short), короткое название гостевой команды (away_short).
Отсортируйте запрос по возрастанию id матча.

In [ ]:
SELECT
    m.id "id матча",
    h.short_name "домашняя команда",
    a.short_name "гостевая команда"
FROM
    sql.matches m
    JOIN sql.teams h on m.home_team_api_id = h.api_id
    JOIN sql.teams a on m.away_team_api_id = a.api_id
ORDER BY 
    "id матча"

## Фильтрация и агрегатные функции <a class="anchor" id=4></a>

[к содержанию](#0)

### РАБОТА С ОБЪЕДИНЁННЫМИ ТАБЛИЦАМИ

Принцип построения запроса и порядок операторов такой же, как и с обычной таблицей.

Вспомним его:



In [ ]:
SELECT... 
FROM... 
WHERE... 
GROUP BY... 
ORDER BY... 
LIMIT...

К соединённым таблицам применимы функции фильтрации данных.

Например, можно вывести `id` матчей, в которых команда `Arsenal` была гостевой.

In [ ]:
SELECT 
    m.id /*столбец id таблицы m*/
FROM
    sql.teams t /*таблица teams с алиасом t*/
    JOIN sql.matches m ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица matches с алиасом m; условие: away_team_api_id таблицы m равен api_id таблицы t*/
WHERE long_name = 'Arsenal' /*long_name таблицы teams имеет значение Arsenal*/

>Принципиальное отличие фильтрации данных по соединённым таблицам от аналогичного действия по одиночным таблицам заключается в том, что, фильтруя записи одной таблицы, мы также будем фильтровать и записи другой таблицы, поскольку соединённые на уровне запроса таблицы по сути **являются единой таблицей**.

In [ ]:
SELECT 
	m.id id_1,
	m.season,
	t.id id_2,
	t.long_name
FROM
	sql.teams t
JOIN sql.matches m ON m.away_team_api_id = t.api_id

Результат этого запроса условно делиться на дву части

<img src=sql_3_img2.png>

Одна часть — таблица `matches` с алиасом `m`, вторая — `teams` с алиасом `t`, но после соединения они являются одной таблицей.

Таким образом, если вы отфильтруете данные по одной части таблицы, то другая, соединённая, часть пропадёт вместе с ней.

Напишите запрос, который выведет полное название команды (`long_name`), количество голов домашней команды (`home_goal`) и количество голов гостевой команды (`away_goal`) в матчах, где домашней командой были команды с коротким названием `GEN`.
Отсортируйте запрос по `id` матча в порядке возрастания.

In [ ]:
SELECT
    t.long_name,
    m.home_team_goals,
    m.away_team_goals
FROM
	sql.teams t
JOIN sql.matches m ON m.home_team_api_id = t.api_id
WHERE t.short_name = 'GEN'
ORDER BY m.id

Также мы можем отфильтровать записи сразу по двум таблицам.

In [ ]:
SELECT * /*выбор всех полей*/
FROM    
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t on t.api_id = m.home_team_api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: home_team_api_id таблицы m равен api_id таблицы t*/
WHERE
    t.short_name = 'GEN' /*столбец short_name таблицы t имеет значение GEN*/
    AND m.season = '2008/2009' /*столбец season таблицы m имеет значение 2008/2009*/

Напишите запрос, чтобы вывести `id` матчей, короткое название домашней команды (`home_short`), короткое название гостевой команды (`away_short`) для матчей сезона `2011/2012`, в которых участвовала команда с названием `Liverpool`.
Отсортируйте по `id` матча в порядке возрастания.

In [ ]:
SELECT
    m.id,
    t.short_name home_short,
    t_a.short_name away_short
FROM sql.matches m
JOIN sql.teams t on t.api_id = m.home_team_api_id
JOIN sql.teams t_a on t_a.api_id = m.away_team_api_id
WHERE m.season = '2011/2012' and (t.long_name = 'Liverpool' or t_a.long_name = 'Liverpool')
ORDER BY m.id

### АГРЕГАЦИЯ ДАННЫХ

К соединённым таблицам также применимы любые агрегатные функции — самые важные функции для анализа данных.

Например, мы можем вывести сумму голов по командам для матчей, где команда выступала в гостях.

In [ ]:
SELECT
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY t.id /*группировка по столбцу id таблицы t*/

>**Обратите внимание!** В данном запросе была использована группировка по столбцу `id` таблицы `teams`, хотя этот столбец не выводится в запросе. Это необходимо для того, чтобы команды с одинаковым названием, если такие найдутся, не группировались между собой. Группировка по названию команды в данном запросе будет неверной, так как есть несколько команд с одинаковым полным названием — мы говорили об этом в начале модуля.

Также, применяя агрегатные функции к соединённым таблицам, обращайте внимание на указание алиасов (или таблиц) при группировке и указании столбцов агрегатных функций. В нашей соединённой таблице есть два столбца с названием `id`, и если бы мы сформировали запрос без указания таблицы, как указано ниже, то...

In [ ]:
SELECT
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY id /*группировка по столбцу id, но не указано, какой таблицы*/

система выдала бы уже знакомую нам ошибку: "`... column "id" is ambiguous ...`".

Мы можем использовать оператор `HAVING` для фильтрации сгруппированных данных.

Поставим задачу — вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.

In [ ]:
SELECT
    m.season, /*столбец season таблицы m*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM sql.matches m /*таблица matches с алиасом m*/
GROUP BY m.season /*группировка по столбцу season таблицы m*/

Затем добавим таблицу с командами, группировку по командам и условие фильтрации.

In [ ]:
SELECT
    m.season, /*столбец season таблицы m*/
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM sql.matches m /*таблица matches с алиасом m*/
JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY m.season, t.id /*группировка по столбцам season таблицы m и id таблицы t*/
HAVING SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100 /*оператор фильтрации сгруппированных данных; функция суммирования; home_team_goals таблицы m; функция суммирования; away_team_goals таблицы m; больше 100*/

### ВНИМАНИЕ!
Напишите запрос, с помощью которого можно вывести список полных названий команд, сыгравших в гостях `150` и более матчей.
Отсортируйте список по названию команды.

In [ ]:
SELECT
    t.long_name
FROM
    sql.matches m
JOIN sql.teams t ON m.away_team_api_id = t.api_id /* всё для гостевых команд */
GROUP BY t.id /* группируем по id матча ХЗ почему, я бы группировал по имени команды */
HAVING COUNT(*) >= 150 /* оставляем только позиции встретившиеся более 150 раз */
ORDER BY t.long_name

## Способы соединения таблиц <a class="anchor" id=5></a>

[к содержанию](#0)

### INNER JOIN

`INNER JOIN` — это тот же `JOIN` (слово `inner` в операторе можно опустить).

<img src=sql_3_gif1.gif>

>Для `INNER JOIN` работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

Рассмотрим на примере нашего датасета. 

В таблице `teams` есть данные о 299 различных командах — можем проверить это с помощью запроса.


In [ ]:
SELECT 
COUNT(DISTINCT id)
FROM sql.teams

Теперь добавим к `teams` таблицу с матчами.

In [ ]:
SELECT 
COUNT(DISTINCT t.id) /*оператор подсчёта строк; оператор исключения повторяющихся строк; столбец id*/
FROM 
sql.teams t /*таблица teams с алиасом t*/
/*оператор соединения inner JOIN; таблица teams с алиасом t; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id 

И в таблице останется уже не `299` команд, а только `292`.

**?Почему?**

Дело в том, что таблица `sql.matches` по какой-то причине не содержит информацию о командах `Lierse SK`, `KVC Westerlo`, `KAS Eupen`, `Club Brugge KV`, `KV Oostende`, `RSC Anderlecht` и `Hull City`, зато они есть в таблице `sql.teams`. Возможно, эти команды не участвовали ни в одном матче или записи по этим матчам были удалены.

### LEFT OUTER JOIN И RIGHT OUTER JOIN <a class="anchor" id=5-1></a>

[к содержанию](#0)

Также существуют схожие друг с другом типы соединения — `LEFT JOIN` и `RIGHT JOIN` (слово `outer` в операторе можно опустить).

<img src=sql_3_gif2.gif>

>Для `LEFT JOIN` работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение `NULL`.

`LEFT JOIN` может быть полезен, когда соответствующих записей во второй таблице может не быть, но важно сохранить записи из первой таблицы.

Поставим следующую задачу: вывести полные названия команд, данных по которым нет в таблице matches.

Для начала посмотрим на результат запроса после соединения.

In [ ]:
SELECT
    t.long_name, /*столбец long_name таблицы t*/
    m.id /*столбец id таблицы m*/
FROM sql.teams t /*таблица teams с алиасом t*/
/*оператор соединения left JOIN; таблица matches с алиасом m; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id 
ORDER BY m.id DESC /*сортировка по id таблицы m по убыванию, чтобы увидеть строки со значением null*/

>**Вывод**: в таблице `teams` сохранились все записи, а в таблице matches есть пустые строки.

<img src=sql_3_img3.png>

Теперь, чтобы выбрать такие команды, которые не принимали участия в матчах, достаточно добавить условие `where m.id is null` (или любое другое поле таблицы matches).

In [ ]:
SELECT
    t.long_name /*столбец long_name таблицы t*/
FROM 
    sql.teams t /*таблица teams с алиасом t*/
/*оператор соединения left JOIN; таблица matches с алиасом m; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id 
WHERE m.id IS NULL /*условие: столбец id таблицы m имеет значение null*/

>**Обратите внимание!** Если мы добавим какой-либо фильтр по значению для таблицы `matches`, то `LEFT JOIN` превратится в `INNER JOIN`, поскольку для второй таблицы станет необходимым присутствие значения в строке.

In [ ]:
SELECT
    t.long_name 
FROM sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
WHERE
    m.season = '2008/2009'
AND t.long_name = 'KAS Eupen'

>В ответе `Metabase` получим `No results!`, так как вместе с фильтром `m.season = '2008/2009'` исчезли все строки, значения которых `NULL`.

Используя `LEFT JOIN`, выведите список уникальных названий команд, содержащихся в таблице `matches`. Отсортируйте список в алфавитном порядке.
В поле ниже введите запрос, с помощью которого вы решили задание.

In [ ]:
SELECT
    DISTINCT(t.long_name) 
FROM sql.matches m 
LEFT JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
ORDER BY 1 

С `LEFT JOIN` также работают агрегатные функции, что позволяет не потерять значения из левой таблицы. Например, мы можем вывести сумму голов команд по гостевым матчам.

In [ ]:
SELECT
    t.long_name,
    SUM(m.away_team_goals) total_goals
FROM   
    sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id
GROUP BY t.id
ORDER BY 2 DESC /* NULLS FIRST или NULLS LAST - не забывай */ 

>**Обратите внимание!** При применении функций `SUM`, `MIN`, `MAX`, `AVG` к полям со значением `NULL` в результате получится `NULL`, а не `0`. А при использовании функции `COUNT`, наоборот, получится `0`.

Используя `LEFT JOIN`, напишите запрос, который выведет полное название команды (`long_name`), количество матчей, в которых участвовала команда, — домашних и гостевых (`matches_cnt`).
Отсортируйте по количеству матчей в порядке возрастания, затем по названию команды в алфавитном порядке.

In [ ]:
select
t.long_name,
count(m.id)
from
sql.teams t
left join sql.matches m on t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
group by t.id
order by 2, 1

>При использовании `RIGHT JOIN` сохраняется та же логика, что и для `LEFT JOIN`, только за основу берётся правая таблица.
>
>Чтобы из `LEFT JOIN` получить `RIGHT JOIN`, нужно просто поменять порядок соединения таблиц.

Вообще, применение `RIGHT JOIN` считается **плохим тоном**, так как язык `SQL` читается и пишется слева направо, а такой оператор усложняет чтение запросов.

### FULL OUTER JOIN <a class="anchor" id=5-2></a>

[к содержанию](#0)

Оператор `FULL OUTER JOIN` объединяет в себе `LEFT` и `RIGHT JOIN` и позволяет сохранить кортежи обеих таблиц. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

`FULL OUTER JOIN` может быть полезен в ситуациях, когда схема данных недостаточно нормализована и не хватает таблиц-справочников.

>**Пример**: в базе данных интернет-магазина есть две таблицы — с зарегистрированными пользователями и пользователями, оформившими заказ. При этом оформить заказ можно без регистрации, а зарегистрироваться — без оформления заказа.

Предположим, что вам необходимо получить полный список пользователей — и оформивших заказ, и зарегистрированных, — но в базе данных этой объединённой таблицы нет. В данном случае можно использовать `FULL OUTER JOIN` для получения полного списка, соединив таким образом таблицы c заказами и регистрациями по `id` пользователя.

<img src=sql_3_img4.png>

Синтаксис `FULL OUTER JOIN` аналогичен другим `JOIN`.

In [ ]:
SELECT 
…
FROM
	table1
FULL OUTER JOIN table2 ON условие

### CROSS JOIN <a class="anchor" id=5-3></a>

[к содержанию](#0)

На самом деле с этим оператором соединения таблиц вы познакомились в самом начале текущего модуля.

>`CROSS JOIN` соединяет таблицы так, что каждая запись в первой таблице присоединяется к каждой записи во второй таблице, иначе говоря, даёт декартово произведение.

<img src=sql_3_img5.png>

В начале модуля вы использовали этот оператор, записывая таблицы через запятую.

In [ ]:
SELECT *
FROM
    sql.teams,
    sql.matches
    
/* равносилен */
    SELECT *
FROM
    sql.teams
    CROSS JOIN sql.matches

>**Обратите внимание!** Условие для `CROSS JOIN`, в отличие от других операторов, не требуется.

Также этот запрос можно записать с помощью `INNER JOIN` с условием `on true` — результат будет тот же.

In [ ]:
SELECT * /*выбор всех полей*/
FROM
    sql.teams /*таблица teams*/
    JOIN sql.matches ON TRUE /*оператор соединения INNER JOIN; таблица matches; условие: для всех случаев*/

`CROSS JOIN` может быть полезен, когда необходимо создать таблицу фактов.

Например, с помощью такого запроса мы можем получить все возможные комбинации полных названий команд в матчах.

In [ ]:
SELECT
    DISTINCT /*оператор исключения повторяющихся строк*/
    t1.long_name home_team, /*столбец long_name таблицы t1; новое название*/
    t2.long_name away_team /*столбец long_name таблицы t2; новое название*/
FROM
    sql.teams t1 /*таблица teams с алиасом t1*/
    CROSS JOIN sql.teams t2 /*оператор соединения CROSS JOIN; таблица teams с алиасом t2*/

Напишите запрос, который выведет все возможные уникальные комбинации коротких названий домашней команды (`home_team`) и коротких названий гостевой команды (`away_team`).
Отсортируйте запрос по первому и второму столбцам.

In [ ]:
SELECT
    DISTINCT
    t1.short_name,
    t2.short_name
FROM
    sql.teams t1
    CROSS JOIN sql.teams t2
ORDER BY 1, 2

### NATURAL JOIN <a class="anchor" id=5-4></a>

[к содержанию](#0)

Ключевое слово `natural` в начале оператора `JOIN` позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.

`NATURAL JOIN` можно использовать с любыми видами соединений, которые требуют условия соединения:

* `NATURAL INNER JOIN` (возможна запись `NATURAL JOIN`);
* `NATURAL LEFT JOIN`;
* `NATURAL RIGHT JOIN`;
* `NATURAL FULL OUTER JOIN`.

При использовании `NATURAL JOIN` прежде всего стоит обратить внимание на ключи таблиц. Для наших таблиц `teams` и `matches` этот вид соединения не подойдёт, так как общим для обеих таблиц является столбец `id`, но таблицы соединяются по другим столбцам.

Когда у таблиц есть несколько столбцов с одинаковыми именами, при `NATURAL JOIN` условие соединения будет применено на все столбцы с одинаковыми именами.

То есть для таблиц `table1` и `table2`

`table1: id, name, ...`

`table2: id, name, ...`

запрос


In [ ]:
SELECT 
…
FROM 
    table1 
NATURAL JOIN table2

/* равносилен */

SELECT
…
FROM 
    table1 t1
INNER JOIN table2 t2 ON t1.id = t2.id AND t1.name = t2.name

### ОБЩАЯ ЛОГИКА ПОСТРОЕНИЯ ЗАПРОСА С JOIN И ВИДЫ JOIN'OB<a class="anchor" id=5-5></a>

[к содержанию](#0)

При построении запроса с несколькими `JOIN` старайтесь идти слева направо. Сначала выберите таблицу, которая является центральной в соответствии с поставленной задачей, вопросом. Затем добавляйте таблицы поэтапно в зависимости от бизнес-логики запроса.

Например, для ответа на вопрос: «Какая команда сыграла больше всех матчей в сезоне 2010/2011?» в качестве центральной лучше выбрать таблицу с командами.

А для ответа на вопрос: «В каком сезоне участвовало больше всего команд?» — таблицу с матчами.

Стоит отметить, что из рассмотренных видов соединений чаще всего используются `INNER JOIN` и `LEFT JOIN`. Другие операторы используются реже, но стоит помнить об их существовании при решении нестандартных задач.
 
>`CROSS JOIN` соединяет таблицы так, что каждая запись в первой таблице присоединяется к каждой записи во второй таблице ― иначе говоря, даёт декартово произведение.

`INNER JOIN` — это тот же JOIN (слово INNER в операторе можно опустить).

Для INNER JOIN работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

>Ключевое слово `NATURAL` в начале оператора JOIN позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.
>
>`NATURAL JOIN` можно использовать с любыми видами соединений, которые требуют определённого условия:
>
>`NATURAL INNER JOIN` (возможна запись `NATURAL JOIN`);
>`NATURAL LEFT JOIN`;
>`NATURAL RIGHT JOIN`;
>`NATURAL FULL OUTER JOIN`.
>При использовании `NATURAL JOIN` прежде всего стоит обратить внимание на **ключи таблиц**.
>
>Когда у таблиц есть несколько столбцов с одинаковыми именами, при NATURAL JOIN условие соединения будет применено на все столбцы с одинаковыми именами.

Для `LEFT JOIN` работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение NULL.

`LEFT JOIN` полезен, когда соответствующих записей во второй таблице может не быть, но важно сохранить записи из первой таблицы.

Чтобы из `LEFT JOIN` получить RIGHT JOIN, нужно просто поменять порядок соединения таблиц.

>Для `RIGHT JOIN` работает следующее правило: из правой (относительно оператора) таблицы сохраняются все строки, а из левой добавляются только те, которые соответствуют условию соединения. Если в левой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение NULL.
>
>`RIGHT JOIN` полезен, когда соответствующих записей во второй таблице может не быть, но важно сохранить записи из первой таблицы.
>
>Чтобы из `RIGHT JOIN` получить LEFT JOIN, нужно просто поменять порядок соединения таблиц.
>
>**Вообще, применение `RIGHT JOIN` считается дурным тоном, так как язык SQL читается и пишется слева направо, и этот оператор усложняет чтение запросов.**

Оператор `FULL OUTER JOIN` объединяет в себе `LEFT` и `RIGHT JOIN` и позволяет сохранить кортежи обеих таблиц. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

`FULL OUTER JOIN` может быть полезен в ситуациях, когда схема данных недостаточно нормализована и не хватает таблиц-справочников.

Пример: в базе данных интернет-магазина есть две таблицы — с зарегистрированными пользователями и пользователями, оформившими заказ. При этом оформить заказ можно без регистрации, а зарегистрироваться — без оформления заказа.

Предположим, что вам необходимо получить полный список пользователей — и оформивших заказ, и зарегистрированных, — но в базе данных этой объединённой таблицы нет. В данном случае можно использовать `FULL OUTER JOIN` для получения полного списка, соединив таким образом таблицы c заказами и регистрациями по id пользователя.

## Итоги <a class="anchor" id=6></a>

[к содержанию](#0)

Напишите запрос, который выведет список уникальных полных названий команд (`long_name`), игравших в гостях в матчах сезона `2012/2013`.
Отсортируйте список в алфавитном порядке.

In [ ]:
SELECT
    DISTINCT
    t.long_name
FROM
    sql.teams t 
JOIN sql.matches m ON t.api_id = m.away_team_api_id
WHERE m.season = '2012/2013'
ORDER BY 1

Напишите запрос, который выведет полное название команды (`long_name`) и общее количество матчей (`matches_cnt`), сыгранных c командой `Inter`, `Inter` была дома.

In [ ]:
SELECT
    t.long_name,
    COUNT(m.id)
FROM
    sql.matches m 
JOIN sql.teams t ON t.api_id = m.away_team_api_id
JOIN sql.teams t_1 ON t_1.api_id = m.home_team_api_id
WHERE t_1.long_name = 'Inter'
GROUP BY t.long_name

Напишите запрос, который выведет полное название команды (`long_name`) и общее количество матчей (`matches_cnt`), сыгранных командой `Inter` в домашних матчах.

In [ ]:
SELECT
    t.long_name,
    COUNT(m.id)
FROM
    sql.teams t
JOIN sql.matches m ON t.api_id = m.home_team_api_id
WHERE t.long_name = 'Inter'
GROUP BY t.long_name

Напишите запрос, который выведет ТОП-10 команд (`long_name`) по суммарному количеству забитых голов в гостевых матчах. Во втором столбце запроса выведите суммарное количество голов в гостевых матчах (`total_goals`).

In [ ]:
SELECT
    t.long_name,
    SUM(m.away_team_goals) total_goals
FROM 
    sql.teams t
JOIN sql.matches m ON t.api_id = m.away_team_api_id
GROUP BY t.long_name
ORDER BY 2 DESC
LIMIT 10

Выведите количество матчей между командами `Real Madrid CF` и `FC Barcelona`.


In [ ]:
SELECT
    COUNT(*)
FROM
    sql.matches m 
JOIN sql.teams t ON t.api_id = m.away_team_api_id
JOIN sql.teams t_1 ON t_1.api_id = m.home_team_api_id
WHERE (t.long_name = 'Real Madrid CF' AND t_1.long_name = 'FC Barcelona')
OR (t_1.long_name = 'Real Madrid CF' AND t.long_name = 'FC Barcelona')

Напишите запрос, который выведет название команды (`long_name`), сезон (`season`) и суммарное количество забитых голов в домашних матчах (`total_goals`).
Оставьте только те строки, в которых суммарное количество голов **менее десяти**.
Отсортируйте запрос по **названию команды**, а затем — **по сезону**.

In [ ]:
SELECT
    t.long_name,
    m.season,
    SUM(m.home_team_goals) total_goals
FROM
    sql.teams t
JOIN sql.matches m ON t.api_id = m.home_team_api_id
GROUP BY t.long_name, m.season
HAVING SUM(m.home_team_goals) < 10
ORDER BY 1, 2

### Охуенно, но..

Все примеры вопросов на собеседоании /взяты из книги [Нолис Ж., Робинсон Э. Data Science для карьериста. СПб.: Питер, 2021.](https://www.litres.ru/zhaklin-nolis/data-science-dlya-karerista/)

#### Вопрос 1 Объясните разницу между левым (LEFT) и внутренним (INNER) соединениями.

**Соединения** — это способы объединения данных из двух разных таблиц (левой и правой) в новую.

Принцип работы соединений заключается в объединении строк между двумя таблицами; набор ключевых столбцов используется для поиска данных в двух одинаковых таблицах, которые необходимо соединить.

В случае левого соединения каждая строка из левой таблицы появляется в полученной таблице, но строки из правой появляются, только если значения в их ключевых столбцах отображаются в левой. При внутреннем соединении обе строки из левой и правой таблиц появляются, только если есть совпадающая строка в другой таблице.

На практике вы можете воспринимать левое соединение как добавление данных из правой таблицы к левой, если она существует (например, с использованием правой таблицы в качестве подстановочной). Внутреннее соединение больше похоже на поиск всех общих данных и создание новой таблицы только из пар.

Обратите внимание, что в нашем варианте ответа мы не упоминали какие‐либо сложности, связанные с появлением повторяющихся строк в данных. Возможно, о них стоит упомянуть, потому что повторяющиеся строки могут повлиять на результат, но с большей долей вероятности эта информация может увести вас от мысли, которую вы пытаетесь донести до интервьюера.

#### Вопрос 2 Перед вами школьная таблица TABLE_A с оценками от 0 до 100, выставленными ученикам нескольких классов. 

<img src=sql_3_img6.png>



In [ ]:
SELECT 
  CLASS, 
  MAX(GRADE) INTO TABLE_B /* записать новую таблицу как TABLE_B */
FROM 
  TABLE_A 
GROUP BY 
  CLASS

У соискателей этот вопрос обычно вызывает трудности. Одна из причин — непонимание, что именно нужно группировать (в данном случае переменную класса). Другая — в том, что вопрос кажется настолько простым, что соискатели начинают слишком усложнять ответ и упускают простое решение.

Если вы услышали вопрос, который кажется вам слишком лёгким, вполне возможно, что таким он и является.

К слову, строка `INTO TABLE_B` была совершенно необязательной, но она хорошо подготовит вас к следующему вопросу.

#### Вопрос 3 Рассмотрим таблицу из предыдущего вопроса.

<img src=sql_3_img6.png>

Как найти не только самую высокую оценку в каждом классе и учащегося, получившего её?


In [ ]:
SELECT 
  a.CLASS, 
  a.GRADE, 
  a.STUDENT 
FROM 
  TABLE_A a 
  INNER JOIN TABLE_B b ON a.CLASS = b.CLASS 
  AND a.GRADE = b.GRADE

Этот запрос выбирает всех учащихся и их оценки из исходной таблицы `TABLE_A`, где указаны классы и оценки, которые отображаются в таблице максимальных значений `TABLE_B`. Внутреннее соединение работает как фильтр, сохраняя только максимальные комбинации классов/оценок, потому что только в этом случае оценка появляется в таблице `TABLE_B`.

В качестве альтернативы можно было бы использовать **подзапрос**, чтобы сделать то же самое, не вызывая таблицу `TABLE_B`:

In [ ]:
SELECT 
  a.CLASS, 
  a.GRADE, 
  a.STUDENT 
FROM 
  TABLE_A a 
  INNER JOIN (
    SELECT 
      CLASS, 
      MAX(GRADE) 
    FROM 
      TABLE_A 
    GROUP BY 
      CLASS
  ) b ON a.CLASS = b.CLASS 
  AND a.GRADE = b.GRADE

Как видите, эту задачу можно решить несколькими способами, но любое решение почти наверняка требует более одного запроса из таблицы `TABLE_A`, поэтому этот вопрос может легко поставить в тупик. На бумаге решение может показаться лёгким, но додуматься до него во время интервью может быть непросто. Даже если вы ответили неправильно, это не значит, что вы провалились.

Решение не даёт каких‐либо специфичных случаев для привязки к максимальному значению. В примере будут возвращены строки для нескольких студентов. Возможно, стоит указать на этот факт интервьюеру, потому что так вы показываете, что обращаете внимание на пограничные случаи.